In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 4.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 48.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 6.8 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-xji3sd82/overlay/local/bin
    changing mode of /tmp/pip-build-env-xji3sd82/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-xji3sd82/overlay/lo

In [2]:
# For download the models
!pip install huggingface_hub

Выбирем лучшую модель из тех, что можем себе позволить



![asd](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc24dac6d-6b5e-4b5f-938c-05951c938a9e_1085x543.png)






# Кванторизация

Мы можем сами сквантизировать модель, но лучше использовать уже преквантизированну:


```
# берем оригинальную LLAMA, кладем в ./models
ls ./models

# скачиваем зависимости
python3 -m pip install -r requirements.txt

# конвертируем 7B модель в ggml FP16 формат
python3 convert.py models/7B/

# кванторизуем до 4 бит
./quantize ./models/7B/ggml-model-f16.bin ./models/7B/ggml-model-q4_0.bin q4_0

# запускаем получение предсказания
./main -m ./models/7B/ggml-model-q4_0.bin -n 128
```



На huggingface большой список кванторизированных моделей, мы возьмем [llama-2-7b-chat-GGML](https://huggingface.co/TheBloke/Llama-2-7B-chat-GGML)

In [3]:
model_name_or_path = "TheBloke/Llama-2-7B-chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q5_1.bin" # the model is in bin format

Скачиваем

In [4]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-7b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Инициализируем модель

In [5]:
# GPU
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=43,
    n_ctx=4096, # Максимальное количество токенов на входе
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


Для задачи выделения плюсов и минусов продукта мы подобрали методом проб и ошибок следующий промпт

In [39]:
prompt = "Write down from 3 to 6 advantages and disadvantages of the product based on its reviews. "+ \
        "Reviews are the following:"

review = '''
Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process.
'''

prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}
 {review}

ASSISTANT:
'''

Генерируем ответ

In [41]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=256,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USER:'], # Останавливаемся, если увидели специальный токен "USER:"
    echo=True # возвращаем ответ вместе с промптом
)

print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write down from 3 to 6 advantages and disadvantages of the product based on its reviews. Reviews are the following:
 
Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process.


ASSISTANT:
Thank you for bringing this product review to my attention. Based on the information provided, here are some advantages and disadvantages of the product: 
Advant

In [11]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [30]:
from datasets import load_dataset

dataset = load_dataset("SetFit/amazon_reviews_multi_en", split='train')

In [32]:
dataset

Dataset({
    features: ['id', 'text', 'label', 'label_text'],
    num_rows: 200000
})

In [33]:
from tqdm.notebook import tqdm

In [35]:
product_reviews = {}

In [36]:
for elem in tqdm(dataset, total=len(dataset)):
    if elem['id'] not in product_reviews.keys():
        product_reviews[elem['id']] = [elem['text']]
    else:
        product_reviews[elem['id']].append(elem['text'])

  0%|          | 0/200000 [00:00<?, ?it/s]

In [38]:
dataset[0]['text']

"Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process."